In [ ]:
%matplotlib inline

import json
import os
import matplotlib.pyplot as plt
from extras import foldGeneration
import numpy as np
from ModeTonicEstimation import BozkurtEstimation
from ModeTonicEstimation import Evaluator as ev

We have out data set in "./data". Her klasörün içinde o makama ait kayitlarin ppitch trackleri var. (copyright issuelari dolayisiyla kayitlari veremiyoruz.) Bunu hesaplamak icin extras'taki kodu kullandik. Ama ayni formatta kaydetmek kaydiyla (tek sutunda pitch degerleri), istediginiz melody extraction algoritmasini kullanabilirsiniz.

In [ ]:
demo_dir = 'demo'
data_dir = os.path.join(demo_dir, 'data')
annotation_file = os.path.join(data_dir, 'annotations.json')

be = BozkurtEstimation.BozkurtEstimation(step_size=7.5, smooth_factor=15, chunk_size=0, frame_rate=128/44100)
evaluator = ev.Evaluator()

datayi foldlara ayiriyoruz, ki denemelerimiz data dependent olsun. demoda az sayida kayit oldugu icin 5 fold kullaniyoruz. Isterseniz makam_recognition_test_dataset'in tamamini indirip bunu deneyebilirsiniz.

In [ ]:
with open(annotation_file, 'r') as a:
    annotation = json.load(a)
modes = set([m['makam'] for m in annotation])

In [ ]:
n_folds = 5
folds = foldGeneration.stratified_fold(data_dir, annotation_file, n_folds)

# we will use the first fold only
fold = folds['fold0']

mbids

In [ ]:
models = dict()
for cur_mode in modes:
    print "Training: " + cur_mode
    [file_list, tonic_list] = zip(*[(rec['file'], rec['tonic']) for rec in fold['train'] 
                                    if rec['mode'] == cur_mode])
    models[cur_mode] = be.train(cur_mode, file_list, tonic_list, metric='pcd')

In [ ]:
fig, ax = plt.subplots()
for cur_mode in modes:
    ax.plot(models[cur_mode].bins, models[cur_mode].vals, label=cur_mode)

legend = ax.legend(loc='upper right')    
    
plt.xlabel('Normalized Pitch Class (cents)')
plt.title('Pitch Class Distributions of the Makams')
plt.grid(True)
    
ax.get_yaxis().set_visible(False)
plt.show()

From the distributions you can see some important differences in the peak locations, heights and shapes. In broder sense, these aspects of the distributions convey information about how the notes are being performed. For instance the 3rd degree of Hicaz makam (i.e. the third peak) does not coincide with the 3rd degrees of Saba and Hüseyni. Similarly the third degree of Saba makam is performed more frequent than the third of Hüseyni makam.

Using these distributions for comparison, we can find out the makam and/or tonic of another recording.

First let's load the pitch tracks for all the testing recordings. This is needed for all the estimation tasks below.

We start with tonic identification. Here, the assumption is given a recording with known makam and unknown tonic, the peaks of the makam distribution and the peaks of the distribution of the recording should coincide. We can check this by shifting the distribution of the template and finding the best shift according to a distance/similarity metric. The best shift will simply give us the tonic frequency.

In [ ]:
# Tonic Identification
for rec in fold['test']:
    res = be.estimate(rec['file'], mode_in=models[rec['mode']], distance_method="bhat")[0][0]
    
    # evaluate
    e = evaluator.tonic_evaluate(rec['mbid'], res, rec['tonic'])
    
    print(rec['mbid'] + ", Anno: " + ("%.1f" % rec['tonic']) + " Hz, "
          "Est: " + ("%.1f" % res) + ' Hz: ' + str(e['tonic_eval']))

So, for these 3 recordings, we identified all the tonics correctly! Note that this is a rather small and easy experiment (only 3 recordings, 3 classes etc.), so don't expect a "perfect" result for real datasets. 

Also, you might have noticed that the estimated tonic frequencies have octave errors. This is because of the octave we restrict ourselves when computing the PCDs. In this case by computing the "trend" of the pitch track, one can easily shift the estimated tonic to the correct octave. In practice, we cannot argue a definite tonic octace in many makam performances (esp. instrumental pieces), because all the instruments perform the same (heterophonic) melody in their own register; i.e. each performer has their own tonic frequency, all of which belong to the same pitch class.

Let's continue with makam recognition. If we know the tonic of the recording in advance, we can compare the distribution extracted from the recording with the makam distributions. The closest/most-similar distribution would point the estimated makam.

Tonic identification icin artik bundan daha iyi metodlar var. Hint muzikleri icin Gulati, makamlar icin Sercan. Bu sebeple histogram metodunun en kullanisli oldugu alan artik makam recognition.

In [ ]:
# Makam Recognition
for rec in fold['test']:
    res = be.estimate(rec['file'], mode_in=models, tonic_freq=rec['tonic'],  # first zero is to get the first rank
                                        distance_method="bhat")[0][0]         # second is to get the label
    
    # evaluate
    e = evaluator.mode_evaluate(rec['mbid'], res, rec['mode'])
    
    print(rec['mbid'] + ", Anno: " + rec['mode'] + ", "
          "Est: " + res + ', ' + str(e['mode_eval']))

Again, everything is correct! Apart from the fact that the dataset is small, these three makams are very distinct from each other as explained earlier. You should expect substantial confusion between similar makams (esp. transposed and compound types).

And finally, joint estimation of makam and tonic... Here we know neither the tonic nor the makam of the recording. In this case we basically check every (tonic-candidate, makam) combination. The closest (or the most similar) comparison provides us both the estimated tonic frequency and the makam.

In [ ]:
# Joint Tonic and Makam Estimation
for rec in fold['test']:
    res_mode, res_tonic = be.estimate(rec['file'], mode_in=models, distance_method="bhat")
    res_mode = res_mode[0][0]
    res_tonic = res_tonic[0][0]
    
    # evaluate
    e = evaluator.joint_evaluate(rec['mbid'], [res_tonic, rec['tonic']], [res_mode, rec['mode']])
    
    print(rec['mbid'] + ", Anno: " + rec['mode'] + ", "
              "Est: " + res_mode + ', ' + str(e['mode_eval']))
    print(" " * len(rec['mbid'] + ", ") + "Anno: " + ("%.1f" % rec['tonic']) + " Hz, "
              "Est: " + ("%.1f" % res_tonic) + ' Hz: ' + str(e['tonic_eval']))

Again a perfect result! Note that in a meaningful dataset, this last step will not be so accurate as many makams tend to differ from each other with the location of a single interval, the relative strength (how much they are played compared to others) of some intervals etc. 

Buraya diger dezavantajlari yaz,